File Preprocessing

In [77]:
!pip install -U numpy
!pip install pyldavis

Requirement already up-to-date: numpy in /opt/conda/lib/python3.7/site-packages (1.16.3)


In [123]:
import os
import codecs
import spacy
import numpy
import gensim
from spacy import displacy
import pandas as pd
import itertools as it

We open the preprocessed de file

In [124]:
directory = "../corpus_samples/processed/"
filename = directory + 'de.txt'

with codecs.open(filename, encoding='utf_8') as f:
    sample_page = list(it.islice(f, 1, 2))[0]
    text = f.read()
text

"                   EINLEITUNG.      Historische Bemerkungen über das Wiener und das Potsdamer                     Schweresystem.       Seit der Einführung des STERNEcK'schen Pendelapparats *) haben die relativen Schweremessungen in fast allen der Internationalen Erdmessung angehörenden Staaten die lebhafteste Förderung erfahren. Im Jahre 1900, kaum 10 Jahre nach dem allge- meineren Bekanntwerden dieses Apparats, lag bereits ein Beobachtungsmaterial auf rund 1400 Stationen vor, zu dem das Militär-Geographische Institut in Wien und die öster- reichische Kriegsmarine unter Herrn VON STERNECK% Leitung und persönlicher Mitwirkung die weitaus größten Beiträge geliefert hatten. Eine ausführliche Zusammenstellung und kritische Beleuchtung dieser Arbeiten gab Herr Prof. HELMERT in seinem der XIII. Allge- meinen Konferenz der Internationalen Erdmessung in Paris (1900) vorgelegten Berichte, der außerdem noch etwa 100 g-Werte älteren Ursprungs enthält.**)      Zwischen den Referenz- und Hauptstat

In [125]:
%%time
nlp = spacy.load("de_core_news_md")

CPU times: user 54.2 s, sys: 7.88 s, total: 1min 2s
Wall time: 1min 1s


In [126]:
%%time
doc = nlp(text)

In [127]:
my_stop_words = [u'herr', u'mit',u'der', u'in',u'sein',u'dies',u'einen',u'und', u'ich', u'mit',u'zu']

for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [128]:
tokens = [token for token in doc if not token.is_space and not token.is_stop and not token.is_punct and not token.like_num and not token.is_space]
tokens

[EINLEITUNG,
 Historische,
 Bemerkungen,
 Wiener,
 Potsdamer,
 Schweresystem,
 Einführung,
 STERNEcK'schen,
 Pendelapparats,
 relativen,
 Schweremessungen,
 fast,
 Internationalen,
 Erdmessung,
 angehörenden,
 Staaten,
 lebhafteste,
 Förderung,
 erfahren,
 allge-,
 meineren,
 Bekanntwerden,
 Apparats,
 lag,
 Beobachtungsmaterial,
 Stationen,
 Militär-Geographische,
 Institut,
 Wien,
 öster-,
 reichische,
 Kriegsmarine,
 Herrn,
 STERNECK%,
 Leitung,
 persönlicher,
 Mitwirkung,
 weitaus,
 größten,
 Beiträge,
 geliefert,
 ausführliche,
 Zusammenstellung,
 kritische,
 Beleuchtung,
 Arbeiten,
 Herr,
 Prof.,
 HELMERT,
 XIII,
 Allge-,
 Konferenz,
 Internationalen,
 Erdmessung,
 Paris,
 vorgelegten,
 Berichte,
 g-Werte,
 älteren,
 Ursprungs,
 enthält,
 Referenz-,
 Hauptstationen,
 gesonderten,
 Beobachtungsreihen,
 Form,
 relativen,
 g-Messungen,
 stets,
 darstellen,
 Verbindungen,
 genügender,
 Zahl,
 hergestellt,
 Herr,
 Prof.,
 HELMERT,
 Reihen,
 näherungsweise,
 einheitliches,
 Schweresyst

In [60]:
token_text = [token.orth_ for token in tokens]
token_text
token_pos = [token.pos_ for token in tokens]
token_pos
token_ent_type = [token.ent_type_ for token in tokens]
token_ent_type
pd.DataFrame(zip(token_text, token_ent_type), columns= ['text', 'ent type'])

,text,ent type
0,EINLEITUNG,MISC
1,Historische,
2,Bemerkungen,
3,Wiener,MISC
4,Potsdamer,MISC
5,Schweresystem,
6,Einführung,
7,STERNEcK'schen,MISC
8,Pendelapparats,MISC
9,relativen,


In [185]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [186]:
intermediate_directory = '../corpus_samples/lemmatized/'
def discardable(token):

    print(token.text)
    print(token.is_stop)
    return token.is_punct or token.is_space or token.is_stop or token.is_punct or token.like_num

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not discardable(token)])


unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all_de.txt')

with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(filename):
        f.write(sentence + '\n')

/opt/conda/lib/python3.7/runpy.py:193: DeprecationWarning: [W016] The keyword argument `n_threads` on the is now deprecated, as the v2.x models cannot release the global interpreter lock. Future versions may introduce a `n_process` argument for parallel inference via multiprocessing.
  "__main__", mod_spec)


                                  
False
2
False
        
False
auf
True
der
True
Sternwarte
False
in
True
Wien
False
(
False
1884
False
)
False
,
False
die
True
andere
True
durch
True
VON
True
Oarr
False
auf
True
der
True
Sternwarte
False
   
False
in
True
München
False
(
False
1877
False
)
False
ausgeführt
False
worden
True
war
True
.
False
*
False
)
False
Das
True
durch
True
die
True
Konstanten
False
                                                               
False
g
False
(
False
val
False
.
False
abs
False
.
False
)
False
        
False
Wien
False
,
False
M.-G.
False
I.
False
 
False
48
False
°
False
121.7
False
16
False
°
False
211.5
False
 
False
183
False
m
False
   
False
980.876
False
cm
False
sek-2
False
         
False
definierte
False
„
False
Wiener
False
g-System
False
"
False
ist
True
bisher
True
allgemein
False
als
True
Grundlage
False
für
True
die
True
relativen
False
   
False
g-Messungen
False
beibehalten
False
worden
True
,
False
und
True
die
True
näherungsweise

Vorbereitung
False
weiterer
False
umfassenderer
False
Unter-
False
suchungen
False
dieser
True
Art
False
Vorversuche
False
anzustellen
False
,
False
welche
True
dem
True
Centralbureau
False
zu
True
übertragen
False
seien
True
.
False
    
False
Es
True
gereicht
False
mir
True
zur
True
besonderen
False
Ehre
False
,
False
Ihnen
True
über
True
diese
True
Vorversuche
False
heute
True
Bericht
False
erstatten
False
zu
True
können
True
.
False
    
False
Von
True
der
True
Erwägung
False
ausgehend
False
,
False
dass
True
ein
True
maassgebendes
False
Urtheil
False
über
True
die
True
Methode
False
und
True
die
True
in
True
Anwendung
False
zu
True
bringenden
False
Instrumente
False
nur
True
durch
True
fortgesetzte
False
Ausführung
False
von
True
Breitenbestimmungen
False
zu
True
erlangen
False
ist
True
,
False
und
True
dass
True
durch
True
derartige
False
Beobachtungen
False
      
False
1
False
)
False
Vergl
False
.
False
Verhandlungen
False
der
True
in
True
Salzburg
False
abgehaltenen
False
Kon

In [187]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [188]:
for unigram_sentence in it.islice(unigram_sentences, 0, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

Sternwarte Wien Oarr Sternwarte München ausführen

Konstante g

val

abs

Wien M.-G. I.

°

°

m

cm

sek-2

definieren

wienern g-System allgemein Grundlage relativ g-Messungen beibehalten näherungsweise ermitteln Reduktionsbeträge Beobachtungsreihen Herrn Professor HELMERT dienen System angepaßt Normalformel ableiten Darstellung Verlauf Schwerkraft Erdoberfläche verwenden

erwähnen berichten S. weisen Herr Professor

HELMERT

wienern Fundamentalkonstante ander Material gestützten Untersuchung

wahrscheinlich Korrektion cm sek-2 bedürfe

alt meist beträchtlich systematisch Fehler behaften absolut Messung Ableitung Normalwertes g eignen erscheinen leiten absolut Bestimmung g Potsdam Weg

ausgedehnt Elimination systematisch Fehler

Bedacht nehmend Arbeit Reversionspendeln preußisch Geodätische Institut österreichisch italienisch Reversionspendel verwenden miteinander vergleichen

Resultat Beobachtungsort Potsdam Pendelsaa.1 Geodätische Institut H

g

val

abs

° 22:9

°

4'.1

m

± cm s

In [189]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_de')

In [190]:
bigram_model = Phrases(unigram_sentences)

bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [191]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all_de.txt')
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [192]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [193]:
for bigram_sentence in it.islice(bigram_sentences, 0, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

Sternwarte Wien Oarr Sternwarte München ausführen

Konstante g

val

abs

Wien M.-G. I.

°

°

m

cm

sek-2

definieren

wienern g-System allgemein Grundlage relativ g-Messungen beibehalten näherungsweise ermitteln Reduktionsbeträge Beobachtungsreihen Herrn Professor HELMERT dienen System angepaßt Normalformel ableiten Darstellung Verlauf Schwerkraft Erdoberfläche verwenden

erwähnen berichten S. weisen Herr Professor

HELMERT

wienern Fundamentalkonstante ander Material gestützten Untersuchung

wahrscheinlich Korrektion cm sek-2 bedürfe

alt meist beträchtlich systematisch Fehler behaften absolut Messung Ableitung Normalwertes g eignen erscheinen leiten absolut Bestimmung g Potsdam Weg

ausgedehnt Elimination systematisch Fehler

Bedacht nehmend Arbeit Reversionspendeln preußisch Geodätische Institut österreichisch italienisch Reversionspendel verwenden miteinander vergleichen

Resultat Beobachtungsort Potsdam Pendelsaa.1 Geodätische Institut H

g

val

abs

° 22:9

°

4'.1

m

± cm s

In [194]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

trigram_model = Phrases(bigram_sentences)

trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all_de.txt')


with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
    for bigram_sentence in bigram_sentences:
            
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
        f.write(trigram_sentence + '\n')

In [195]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [196]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all_de.dict')

In [236]:
#extreme filters hyperparams
no_below = 4
no_above = 0.5

trigram_pages = LineSentence(trigram_sentences_filepath)

# learn the dictionary by iterating over all of the pages
trigram_dictionary = Dictionary(trigram_pages)
    
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below, no_above)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [237]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all_de.mm')

In [238]:
def trigram_bow_generator(filepath):
    """
    generator function to read pages from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [239]:
MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

In [240]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all_de')

In [241]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
        
    # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=3,
                           id2word=trigram_dictionary,
                           workers=3)
    
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 40 ms, sys: 260 ms, total: 300 ms
Wall time: 456 ms


In [242]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [243]:
explore_topic(topic_number=2)

term                 frequency

Wien                 0.110
Institut             0.090
Berlin               0.089
Kommission           0.085
Erdmessung           0.072
Herrn                0.066
Gruppe               0.065
Konferenz            0.052
Internationale       0.051
Potsdam              0.047
Dr.                  0.046
S.                   0.041
Professor            0.028
Frage                0.025
cm                   0.020
g                    0.017
Messung              0.015
°                    0.014
Herr                 0.013
wienern              0.012
HELMERT              0.012
absolut              0.011
Schwerkraft          0.010
Bestimmung           0.009


In [244]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [245]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [246]:
pyLDAvis.display(LDAvis_prepared)